In [8]:
import pandas as pd
import numpy as np
import re
import os
path = "data"

In [9]:
#將爬蟲資料重複的去除 並轉換資料型別
def scapy_drop_du(file):
    scrape_data = pd.read_csv(f"{path}/{file}", encoding="cp950", low_memory=False)
    print(scrape_data)
    print(len(scrape_data))
    scrape_drop_du = scrape_data.drop_duplicates()
    print(len(scrape_drop_du))
    scrape_drop_du["tick_name"] = scrape_drop_du["tick_name"].astype(str)
    scrape_drop_du["mounth"] = scrape_drop_du["mounth"].astype(int)
    scrape_drop_du["year"] = scrape_drop_du["year"].astype(int)
    return scrape_drop_du

In [10]:
#將爬取的三個資料合併成一個DataFrame
merge = pd.merge(scapy_drop_du("董監_All.csv"), scapy_drop_du("背書_All.csv"), on=["tick_name", "mounth", "year"],how = "outer")
data = pd.read_csv(f"{path}/本期淨利_All.csv", encoding="cp950", low_memory=False )

#將年份轉乘西元年
merge = pd.merge(data,merge, on=["tick_name",  "year"],how = "outer")
merge["year"] = merge["year"].apply(lambda x : x +1911)


        tick_name  year  mounth 非獨立董事持股設質比例 獨立董事持股設質比例 非獨立監察人持股設質比例  \
0            1101    90       3      70.61%      0.00%       61.23%   
1            1101    90       6      58.13%      0.00%       58.35%   
2            1101    90       9      55.85%      0.00%       60.60%   
3            1101    90      12      56.94%      0.00%       63.92%   
4            1101    91       3      55.22%      0.00%       65.95%   
...           ...   ...     ...         ...        ...          ...   
105364       9962   109       3       0.00%      0.00%        0.00%   
105365       9962   109       6       0.00%      0.00%        0.00%   
105366       9962   109       9       0.00%      0.00%        0.00%   
105367       9962   109      12       0.00%      0.00%        0.00%   
105368       9962   110       3       0.00%      0.00%        0.00%   

       獨立監察人持股設質比例 非獨立董監持股設質比例 獨立董監持股設質比例 全體董監持股設質比例    全體董監持股合計  大股東股數  
0            0.00%      65.09%      0.00%     65.09%   168722989    0.0  

C:\Users\kobe\AppData\Local\conda\conda\envs\pandas\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\kobe\AppData\Local\conda\conda\envs\pandas\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\kobe\AppData\Local\conda\conda\envs\pandas\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

112276


In [11]:
#讀取兩個財報資料
sherry_data_1 = pd.read_csv(f"{path}/2001_2010.csv", encoding="cp950", sep="\t", low_memory=False)
sherry_data_2 = pd.read_csv(f"{path}/2011_2021.csv", encoding="cp950", sep="\t", low_memory=False)

#concat兩個財報資料
sherry_data = pd.concat([sherry_data_1,sherry_data_2],ignore_index=True)


In [12]:
sherry_data["年/月"]  = sherry_data["年/月"].apply(lambda x : str(x) if not re.match(r"(....)(..?)*", str(x)) == None else np.nan)
#將沒有年份的刪除
sherry_data.dropna(inplace=True)

In [13]:
#將年月分開成兩個欄位
sherry_data[["year", "mounth"]] = sherry_data["年/月"].str.extract(r"(....)(..?)")

#將資料型別對應爬取資料 以利合併
sherry_data["year"] = sherry_data["year"].astype(int)
sherry_data["mounth"] = sherry_data["mounth"].astype(int)

#公司欄位有空白 移除空白
sherry_data["公司"] =  sherry_data["公司"].str.strip()

In [14]:
#用(股票id,年,月) 合併資料
merge_data = pd.merge(sherry_data, merge, left_on=["公司", "mounth", "year"], right_on=["tick_name", "mounth", "year"], how="left")

In [15]:
merge_data.head()

,公司,簡稱,上市別,TSE 產業別,最近上市日,證期會代碼,會計月份,統一編號,國際證券編號,電話,...,獨立董事持股設質比例,非獨立監察人持股設質比例,獨立監察人持股設質比例,非獨立董監持股設質比例,獨立董監持股設質比例,全體董監持股設質比例,全體董監持股合計,大股東股數,本公司有背書保證資訊,各子公司有背書保證資訊
0,1101,台泥,TSE,01,1962/02/09,1101,12,ID 11913502,TW0001101004,02-25317099,...,0.00%,63.92%,0.00%,52.04%,0.00%,52.04%,142662064,0.0,"16,407,379","900,000"
1,1102,亞泥,TSE,01,1962/06/18,1102,12,ID 03244509,TW0001102002,02-27338000,...,0.00%,37.89%,0.00%,30.09%,0.00%,30.09%,665231625,0.0,"7,151,267",0
2,1103,嘉泥,TSE,01,1969/11/27,1103,12,ID 11892801,TW0001103000,02-25515211,...,0.00%,0.00%,0.00%,1.93%,0.00%,1.93%,85877418,0.0,"340,000",0
3,1104,環泥,TSE,01,1971/02/02,1104,12,ID 07568009,TW0001104008,02-25077801,...,0.00%,0.00%,0.00%,7.37%,0.00%,7.37%,41432967,0.0,"497,398","82,689"
4,1107,建台,,01,2007/10/22,1107,12,ID 03322504,TW0001107001,07-5666299,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
merge_data.to_csv("data/merge_data.csv",index=False)